# Desgning a Classification Model for the Bank-Notes Dataset Using Different Activation Functions and Varying Epochs

## Submitted By: Ms. Sidra Mehtab

In [1]:
# Importing  the necessary modules
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
import numpy as np
import csv

In [2]:
# Load a CSV file
def loadCsv(filename):
        trainSet = []
        
        lines = csv.reader(open(filename, 'r'))
        next(lines)
        dataset = list(lines)
        for i in range(len(dataset)):
                for j in range(5):
                        #print("DATA {}".format(dataset[i]))
                        dataset[i][j] = float(dataset[i][j])
                trainSet.append(dataset[i])
        return trainSet

In [3]:
def minmax(dataset):
        minmax = list()
        stats = [[min(column), max(column)] for column in zip(*dataset)]
        return stats

In [4]:
# Rescale dataset columns to the range 0-1
def normalize(dataset, minmax):
        for row in dataset:
                for i in range(len(row)-1):
                        row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

In [5]:
# Convert string column to float
def column_to_float(dataset, column):
        for row in dataset:
                try:
                        row[column] = float(row[column])
                except ValueError:
                        print("Error with row",column,":",row[column])
                        pass

In [6]:
# Convert string column to integer
def column_to_int(dataset, column):
        class_values = [row[column] for row in dataset]
        unique = set(class_values)
        lookup = dict()
        for i, value in enumerate(unique):
                lookup[value] = i
        for row in dataset:
                row[column] = lookup[row[column]]
        return lookup

In [7]:
# Find the min and max values for each column

 
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
        dataset_split = list()
        dataset_copy = list(dataset)
        fold_size = int(len(dataset) / n_folds)
        for i in range(n_folds):
                fold = list()
                while len(fold) < fold_size:
                        index = randrange(len(dataset_copy))
                        fold.append(dataset_copy.pop(index))
                dataset_split.append(fold)
        return dataset_split

In [8]:
# Calculate accuracy percentage
def accuracy_met(actual, predicted):
        correct = 0
        for i in range(len(actual)):
                if actual[i] == predicted[i]:
                        correct += 1
        return correct / float(len(actual)) * 100.0

In [9]:
# Evaluate an algorithm using a cross validation split
def run_algorithm(dataset, algorithm, n_folds, *args):
        
        folds = cross_validation_split(dataset, n_folds)
        #for fold in folds:
                #print("Fold {} \n \n".format(fold))
        scores = list()
        Accuracy = list()
        for fold in folds:
                #print("Test Fold {} \n \n".format(fold))
                train_set = list(folds)
                train_set.remove(fold)
                train_set = sum(train_set, [])
                test_set = list()
                for row in fold:
                        row_copy = list(row)
                        test_set.append(row_copy)
                        row_copy[-1] = None
                predicted = algorithm(train_set, test_set, *args)
                actual = [row[-1] for row in fold]
                accuracy = accuracy_met(actual, predicted)
                cm = confusion_matrix(actual, predicted)
                print('\n'.join([''.join(['{:5}'.format(item) for item in row]) for row in cm]))
                #confusionmatrix = np.matrix(cm)
                FP = cm.sum(axis=0) - np.diag(cm)
                FN = cm.sum(axis=1) - np.diag(cm)
                TP = np.diag(cm)
                TN = cm.sum() - (FP + FN + TP)
                print('False Positives\n {}'.format(FP))
                print('False Negetives\n {}'.format(FN))
                print('True Positives\n {}'.format(TP))
                print('True Negetives\n {}'.format(TN))
                TPR = TP/(TP+FN)
                print('Sensitivity \n {}'.format(TPR))
                TNR = TN/(TN+FP)
                print('Specificity \n {}'.format(TNR))
                Precision = TP/(TP+FP)
                print('Precision \n {}'.format(Precision))
                Recall = TP/(TP+FN)
                print('Recall \n {}'.format(Recall))
                Acc = (TP+TN)/(TP+TN+FP+FN)
                print('Áccuracy \n{}'.format(Acc))
                Fscore = 2*(Precision*Recall)/(Precision+Recall)
                print('FScore \n{}'.format(Fscore))
                k=cohen_kappa_score(actual, predicted)
                print('Çohen Kappa \n{}'.format(k))
                scores.append(accuracy)
                Accuracy.append(Acc)
        return Accuracy,scores

In [10]:
# Calculate neuron activation for an input
def activate(weights, inputs):
        activation = weights[-1]
        for i in range(len(weights)-1):
                activation += weights[i] * inputs[i]
        return activation

In [11]:
# Transfer neuron activation
# def transfer_sigmoid(activation):
#        return 1.0 / (1.0 + exp(-activation))

def transfer_leaky_relu(activation):
    if activation > 0:
        return activation
    else:
        return 0.01*activation
    
# def transfer_tanh(activation):
#    t=(np.exp(activation)-np.exp(-activation))/(np.exp(activation)+np.exp(-activation))
#    return t
    

In [12]:
# Forward propagate input to a network output
def forward_propagate(network, row):
        inputs = row
        for layer in network:
                new_inputs = []
                for neuron in layer:
                        activation = activate(neuron['weights'], inputs)
                        neuron['output'] = transfer_leaky_relu(activation)
                        new_inputs.append(neuron['output'])
                inputs = new_inputs
        return inputs

In [13]:
# Calculate the derivative of an neuron output
# def transfer_derivative_sigmoid(output):
#        return output * (1.0 - output)
    
def transfer_derivative_leaky_relu(output):
     return 0.01 if output < 0 else 1
    
# def transfer_derivative_tanh(output): 
#    return 1-output**2

In [14]:
# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
        for i in reversed(range(len(network))):
                layer = network[i]
                errors = list()
                if i != len(network)-1:
                        for j in range(len(layer)):
                                error = 0.0
                                for neuron in network[i + 1]:
                                        error += (neuron['weights'][j] * neuron['delta'])
                                errors.append(error)
                else:
                        for j in range(len(layer)):
                                neuron = layer[j]
                                errors.append(expected[j] - neuron['output'])
                for j in range(len(layer)):
                        neuron = layer[j]
                        neuron['delta'] = errors[j] * transfer_derivative_leaky_relu(neuron['output'])

In [15]:
# Update network weights with error
def update_weights(network, row, l_rate):
        for i in range(len(network)):
                inputs = row[:-1]                
                if i != 0:
                        inputs = [neuron['output'] for neuron in network[i - 1]]
                for neuron in network[i]:
                        for j in range(len(inputs)):
                                temp = l_rate * neuron['delta'] * inputs[j] + mu * neuron['prev'][j]
                                
                                neuron['weights'][j] += temp
                                #print("neuron weight{} \n".format(neuron['weights'][j]))
                                neuron['prev'][j] = temp
                        temp = l_rate * neuron['delta'] + mu * neuron['prev'][-1]
                        neuron['weights'][-1] += temp
                        neuron['prev'][-1] = temp

In [16]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
        for epoch in range(n_epoch):
                for row in train:
                        outputs = forward_propagate(network, row)
                        #print(network)
                        expected = [0 for i in range(n_outputs)]
                        expected[row[-1]] = 1
                        #print("expected row{}\n".format(expected))
                        backward_propagate_error(network, expected)
                        update_weights(network, row, l_rate)

In [17]:
# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
        network = list()
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)], 'prev':[0 for i in range(n_inputs+1)]} for i in range(n_hidden)]        
        network.append(hidden_layer)
        output_layer = [{'weights':[random() for i in range(n_hidden + 1)],'prev':[0 for i in range(n_hidden+1)]} for i in range(n_outputs)]
        network.append(output_layer)
        print(network)
        return network

In [18]:
# Make a prediction with a network
def predict(network, row):
        outputs = forward_propagate(network, row)
        return outputs.index(max(outputs))

In [19]:
# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
        n_inputs = len(train[0]) - 1
        n_outputs = len(set([row[-1] for row in train]))
        network = initialize_network(n_inputs, n_hidden, n_outputs)
        train_network(network, train, l_rate, n_epoch, n_outputs)
        #print("network {}\n".format(network))
        predictions = list()
        for row in test:
                prediction = predict(network, row)
                predictions.append(prediction)
        return(predictions)

In [20]:
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'Bank_Note_Authentication.csv'
dataset = loadCsv(filename)
for i in range(len(dataset[0])-1):
        column_to_float(dataset, i)
# convert class column to integers
column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = minmax(dataset)
normalize(dataset, minmax)
# evaluate algorithm
n_folds = 5
l_rate = 0.1
mu=0.001
# n_epoch = 200
n_epoch = 150
n_hidden = 4
Accuracy, scores = run_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)

# print()
#print(score.0)
#print('Scores: %s' % scores)
#print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))


[[{'weights': [0.17615879778364407, 0.7589167742520844, 0.7392069166892044, 0.5805470101101052, 0.4511136377957703], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.1494295452398231, 0.5039464408606785, 0.5284814960517538, 0.13506734851795632, 0.7614081860479278], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.9888681835793595, 0.21318079766159026, 0.6225476674268445, 0.48040695734353067, 0.11840708649100551], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.8872547085591933, 0.6983487476145435, 0.225027830064511, 0.6352626423450226, 0.8290270470300498], 'prev': [0, 0, 0, 0, 0]}], [{'weights': [0.05003101053142989, 0.17208366606210723, 0.11613748391157386, 0.5632603481367554, 0.5030167815653596], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.6599045536241351, 0.3078631641139429, 0.32764095816192507, 0.7737891750041515, 0.8217239626147195], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.8222080663992034, 0.22026646945301764, 0.743050651434886, 0.280172569160838, 0.6256524546816344], 'prev': [0, 0, 0, 0, 0]}, {'weights

<ipython-input-9-318515d133a9>:37: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)


  151    0
  123    0
False Positives
 [123   0]
False Negetives
 [  0 123]
True Positives
 [151   0]
True Negetives
 [  0 151]
Sensitivity 
 [1. 0.]
Specificity 
 [0. 1.]
Precision 
 [0.55109489        nan]
Recall 
 [1. 0.]
Áccuracy 
[0.55109489 0.55109489]
FScore 
[0.71058824        nan]
Çohen Kappa 
0.0
[[{'weights': [0.9275350630900715, 0.6217922064647944, 0.10554042877522518, 0.9519684937426224, 0.8719968360562562], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.11640322042148132, 0.04057852861539768, 0.7040543505079936, 0.4223945876938293, 0.7272772367764718], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.2534908105250847, 0.625773437289816, 0.8983627317255044, 0.9155832682227135, 0.6169523480152537], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.4149352861665432, 0.35874422113546567, 0.7539069638858403, 0.34123597622677826, 0.7981918357256077], 'prev': [0, 0, 0, 0, 0]}], [{'weights': [0.23805992033281276, 0.6096457728668733, 0.1443734769227446, 0.3414971804099921, 0.11350154050725014], 'prev': 

<ipython-input-9-318515d133a9>:37: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)


  157    0
  117    0
False Positives
 [117   0]
False Negetives
 [  0 117]
True Positives
 [157   0]
True Negetives
 [  0 157]
Sensitivity 
 [1. 0.]
Specificity 
 [0. 1.]
Precision 
 [0.5729927       nan]
Recall 
 [1. 0.]
Áccuracy 
[0.5729927 0.5729927]
FScore 
[0.72853828        nan]
Çohen Kappa 
0.0
[[{'weights': [0.12041025289467455, 0.48827770297803685, 0.266456707118058, 0.12521482370705328, 0.011942322374289605], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.41461703262457617, 0.7990091445015655, 0.6524016192152049, 0.9413497014564688, 0.45879718487022203], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.37701746573464134, 0.5020399898132009, 0.8149426675476388, 0.9179583799504564, 0.15428865680173232], 'prev': [0, 0, 0, 0, 0]}, {'weights': [0.524603425160236, 0.10573512276099672, 0.2545883209913006, 0.46443128136208256, 0.8108713748589025], 'prev': [0, 0, 0, 0, 0]}], [{'weights': [0.7046824748521638, 0.7929380827594594, 0.2283666099438949, 0.6955078645854621, 0.9916502684751451], 'prev': [

<ipython-input-9-318515d133a9>:37: RuntimeWarning: invalid value encountered in true_divide
  Precision = TP/(TP+FP)


  156    3
    0  115
False Positives
 [0 3]
False Negetives
 [3 0]
True Positives
 [156 115]
True Negetives
 [115 156]
Sensitivity 
 [0.98113208 1.        ]
Specificity 
 [1.         0.98113208]
Precision 
 [1.         0.97457627]
Recall 
 [0.98113208 1.        ]
Áccuracy 
[0.98905109 0.98905109]
FScore 
[0.99047619 0.98712446]
Çohen Kappa 
0.9776034003596534
